<a href="https://colab.research.google.com/github/mrarat76/Tensorflow_Practises/blob/main/NLP_Practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## recurrent neural network

## LSTM rnn hücrelerine denir "tanh" non-linearity eklemek için kullnaırız


## Outputta sigmodi kullanıyoruz


NLP'nin amacı bilgiyi doğal dilden elde etmektir( metini veya konuşmayı sıralamak gibi)

NLP genel manada seq2seq problemlerine odaklanır.


**Seq2seq**, kısaltması "sequence to sequence" (diziden diziye) anlamına gelir ve genellikle doğal dil işleme (NLP) ve makine öğrenmesinde kullanılan bir model türüdür. Bu model, bir girdi dizisini (örneğin bir cümle veya kelime dizisi) alır ve başka bir diziye (genellikle başka bir dilde bir cümle veya metin) çevirir. Seq2seq modelleri, özellikle makine çevirisi, otomatik özetleme, soru-cevap sistemleri ve sohbet robotları gibi alanlarda yaygın olarak kullanılır.

Seq2seq modeli tipik olarak iki ana bileşenden oluşur:

Encoder (Kodlayıcı): Girdi dizisini alır ve bu diziyi sabit boyutlu bir vektör temsiline dönüştürür. Bu vektör, girdi dizisinin bir tür özeti olarak düşünülebilir ve genellikle diziye ait önemli bilgileri içerir.

Decoder (Çözücü): Encoder tarafından oluşturulan sabit boyutlu vektörü alır ve hedef diziye dönüştürür. Çözücü, bu vektörü kullanarak adım adım çıktı dizisini üretir.

Bu modeller genellikle tekrarlayan sinir ağları (RNN), uzun kısa süreli bellek ağları (LSTM) veya gated recurrent units (GRU) gibi yapıları kullanır. Son yıllarda, daha gelişmiş seq2seq modelleri, dikkat mekanizmaları (attention mechanisms) ve Transformer mimarileri gibi teknikleri kullanarak daha iyi performans sergilemiştir. Dikkat mekanizması, modelin girdi dizisinin hangi bölümlerine odaklanması gerektiğini daha iyi anlamasını sağlar ve böylece daha doğru çeviriler yapılmasına olanak tanır.







In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9ffa9f42-c4d4-a2f0-647e-2cdbca309ef9)


In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-03-09 10:54:48--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-03-09 10:54:48 (87.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [7]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-03-09 10:54:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.159.207, 142.251.120.207, 142.251.161.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.159.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2024-03-09 10:54:55 (80.6 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [8]:
### Visualizing a text dataset

import pandas as pd

train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

train_df.head()



,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [13]:
## Dataframi shuffellama

train_df_shuffled =train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()



,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [16]:
## How many total samples?


len(train_df), len(test_df)

(7613, 3263)

In [18]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target =row
  print(f"Target:{target}", "(real_diaster)" if target > 0 else "(not real diaster)")
  print(f"Text\n{text}\n")
  print("---\n")



Target:0 (not real diaster)
Text
@CortezEra I dead ass screamed when we signed him I couldn't fucking believe it

---

Target:0 (not real diaster)
Text
@Caitsroberts see U the night wee bArra to get absolutely wrecked ????

---

Target:0 (not real diaster)
Text
Mental/Twitter Note: Make sure my smoke alarm battery is up to snuff at all times or face many twitter reminders of changing my battery.

---

Target:1 (real_diaster)
Text
@hornybigbadwolf -sets the forest on fire-

---

Target:0 (not real diaster)
Text
@wowsavannah what's the harm?? they're collectibles

---

